In [12]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import random
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, Conv2D, MaxPooling2D, Dropout,GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import random

In [13]:
df = pd.read_csv('/kaggle/input/all-texts/text.csv')
df = df.drop_duplicates()
df = df.iloc[:2500]
df['filename'] = None

In [14]:
import pickle
tokenizer = pickle.load(open('/kaggle/input/tokenizer-tx/tokenizer.pkl', 'rb'))

In [15]:
import pickle
new = np.array(pickle.load(open('/kaggle/input/image-data/image_data_6.pkl', 'rb')))[:1000]
y = pickle.load(open('/kaggle/input/target/y_6.pkl', 'rb'))[:1000]
i = 2
newest = np.array(pickle.load(open(f'/kaggle/input/image-data/image_data_{i}.pkl', 'rb')))[:1000]
new = np.concatenate((new, newest), axis=0)
y_n = np.array(pickle.load(open(f'/kaggle/input/target/y_{i}.pkl', 'rb')))[:1000]
y = np.concatenate((y, y_n), axis=0)

In [16]:
dic = {'text':[], 'filename':[], 'is_ad':[]}
for i in range(len(y)):
    dic['text'].append('')
    dic['filename'].append('image')
    dic['is_ad'].append(y[i])
    

In [17]:
df = pd.concat([df, pd.DataFrame(dic)])

In [18]:
inputs_image = Input(shape=(224, 224, 3))
image_model = Conv2D(32, (3, 3), activation='relu')(inputs_image)
image_model = MaxPooling2D((2, 2))(image_model)
image_model = Conv2D(64, (3, 3), activation='relu')(image_model)
image_model = MaxPooling2D((2, 2))(image_model)
image_model = Conv2D(128, (3, 3), activation='relu')(image_model)
image_model = MaxPooling2D((2, 2))(image_model)
image_model = Flatten()(image_model)
image_model = Dense(256, activation='relu')(image_model)
image_model = Dropout(0.5)(image_model)
output = Dense(1, activation='sigmoid')(image_model)

old_image_model = Model(inputs=inputs_image, outputs=output)

In [19]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 8
max_length = 1000

old_text_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(12, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

Adam = 'Adam'
old_text_model.compile(loss='binary_crossentropy',optimizer = Adam, metrics=['AUC'])
text_sequences = tokenizer.texts_to_sequences(df['text'].iloc[:2])
text_data = pad_sequences(text_sequences, maxlen=max_length)
old_text_model.fit(text_data, df.is_ad.iloc[:2], epochs=1, batch_size=32, verbose=1) 

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - AUC: 0.0000e+00 - loss: 0.6915


In [20]:
old_image_model.load_weights('/kaggle/input/image-weights/im_weights.weights.h5')
old_text_model.load_weights('/kaggle/input/old_text/tensorflow2/text_weights/1/tx_weights.weights.h5')

In [21]:
os.environ['PYTHONHASHSEED']=str(1)
tf.random.set_seed(1)
np.random.seed(1)
random.seed(1)
import warnings
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

max_len_text = 1000  
text_sequences = tokenizer.texts_to_sequences(df['text'])
text_data = pad_sequences(text_sequences, maxlen=max_len_text)
max_words = len(tokenizer.word_index) + 1
image_data = []
for filename in df['filename']:
    if filename == 'image':
        continue
    elif filename is not None:
        image_path = os.path.join('/work/dataset', filename)
        img = load_img(image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array /= 255.0  
        image_data.append(img_array)
    else:
        image_data.append(np.zeros((224, 224, 3)))   

image_data = np.array(image_data)
image_data = np.concatenate((image_data, new), axis=0)


inputs_text = Input(shape=(max_len_text,))
inputs_image = Input(shape=(224, 224, 3), name = 'image_0')


embedding_dim = 8  
text_model = Embedding(max_words, embedding_dim, input_length=max_len_text, name = 'text_0')(inputs_text)
text_model = GlobalAveragePooling1D(name = 'text_1')(text_model)
text_model = Dense(200, activation='relu', name = 'text_2')(text_model)
text_model = Dense(100, activation='relu', name = 'text_3')(text_model)
text_model = Dense(50, activation='relu', name = 'text_4')(text_model)
text_model = Dense(24, activation='relu', name = 'text_5')(text_model)
text_model = Dense(12, activation='relu', name = 'text_6')(text_model)


image_model = Conv2D(32, (3, 3), activation='relu', name = 'image_1')(inputs_image)
image_model = MaxPooling2D((2, 2), name = 'image_2')(image_model)
image_model = Conv2D(64, (3, 3), activation='relu', name = 'image_3')(image_model)
image_model = MaxPooling2D((2, 2), name = 'image_4')(image_model)
image_model = Conv2D(128, (3, 3), activation='relu', name = 'image_5')(image_model)
image_model = MaxPooling2D((2, 2), name = 'image_6')(image_model)
image_model = Flatten(name = 'image_7')(image_model)
image_model = Dense(256, activation='relu', name = 'image_8')(image_model)
image_model = Dropout(0.5, name = 'image_9')(image_model)



concatenated = Concatenate()([text_model, image_model])
output = Dense(1, activation='sigmoid')(concatenated)
Adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

model = Model(inputs=[inputs_text, inputs_image], outputs=output)
model.compile(loss='binary_crossentropy', optimizer=Adam, metrics=['accuracy'])

for i in range(len(model.layers)):
    for j in range(10):
        if model.layers[i].name == f'image_{j}':
            model.layers[i].set_weights(old_image_model.layers[j].get_weights())
    for k in range(7):
        if model.layers[i].name == f'text_{k}':
            model.layers[i].set_weights(old_text_model.layers[k].get_weights())

X_text_train, X_text_test, X_image_train, X_image_test, y_train, y_test = train_test_split(
    text_data, image_data, df['is_ad'], test_size=0.2, random_state=42)

with tf.device('/GPU:0'):
    model.fit([X_text_train, X_image_train], y_train, batch_size=32, epochs=10, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate([X_text_test, X_image_test], y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 12s 57ms/step - accuracy: 0.5247 - loss: 1.2007 - val_accuracy: 0.7486 - val_loss: 0.4995
Epoch 2/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.7691 - loss: 0.5223 - val_accuracy: 0.7833 - val_loss: 0.4244
Epoch 3/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.8187 - loss: 0.4084 - val_accuracy: 0.8014 - val_loss: 0.3911
Epoch 4/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.8379 - loss: 0.3632 - val_accuracy: 0.8167 - val_loss: 0.3652
Epoch 5/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.8587 - loss: 0.3041 - val_accuracy: 0.8472 - val_loss: 0.3461
Epoch 6/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.8855 - loss: 0.2854 - val_accuracy: 0.8569 - val_loss: 0.3177
Epoch 7/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.9051 - loss: 0.2378 - val_accuracy: 0.8750 - val_loss: 0.2967
Epoch 8/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.9196 - loss: 0.2051 - val_accuracy: 0.8819 - 

In [24]:
model.save_weights('mm_weights.weights.h5')